In [14]:
from pymilvus import MilvusClient
from langchain.embeddings import OpenAIEmbeddings
from langchain.chains.question_answering import load_qa_chain
from langchain.chat_models import ChatOpenAI
from langchain.schema import Document

import os
os.environ['OPENAI_API_KEY'] = '***'

In [2]:
client = MilvusClient(
    uri="***",
    token="***"
)

In [3]:
# Создаем эмбеддинги из запроса
query = 'Коротко опиши основные правила для банков второго уровня'
embedding = OpenAIEmbeddings()
vector = embedding.embed_query(query)

In [53]:
# х, у - резы поиска по каждому индексу.
x = client.search('a1', [vector], output_fields=['text'])[0]
y = client.search('a2', [vector], output_fields=['text'])[0]
x = [e['entity']['text'] for e in x]
y = [e['entity']['text'] for e in y]
x = [Document(page_content=e) for e in x]
y = [Document(page_content=e) for e in y]

In [57]:
# Соединяем резы
docs = x+y

In [58]:
# Указываем модель и кормим ему резы. Ответ - output
model16k = ChatOpenAI(model='gpt-3.5-turbo-16k')
chain = load_qa_chain(model16k, chain_type="stuff")
output = chain.run(input_documents=docs, question=query)

In [59]:
# See output.txt
output

'Основные правила для банков второго уровня включают следующие требования:\n\n1. Формирование системы управления рисками: банк должен разработать и реализовать систему управления рисками, которая соответствует его бизнес-модели, объему операций и сложности операций. Система управления рисками должна включать процессы выявления, измерения, оценки, мониторинга, контроля и минимизации рисков.\n\n2. Формирование системы внутреннего контроля: банк должен создать систему внутреннего контроля, которая обеспечивает проверку и контроль всех направлений деятельности и бизнес-процессов банка. Система внутреннего контроля должна включать процедуры, методы и полномочия для обеспечения надлежащего контроля и своевременного выявления недостатков.\n\n3. Развитие стратегии управления рисками: банк должен разработать стратегию управления рисками, которая определяет основные цели, направления и сроки реализации. Стратегия должна учитывать риск-аппетит банка и обеспечивать достаточность собственного капит